In [4]:
from PIL import Image
import numpy as np
import cv2

# Paths to the original image and its corresponding mask
original_image_path = '/Users/shinde/Downloads/segmentation_folder/test/images/Image_2_jpg.rf.3dc0ee34a33ac08fb30fc9cb6d971999.jpg'
mask_image_path = '/Users/shinde/Downloads/segmentation_folder/test/masks/Image_2_jpg.rf.3dc0ee34a33ac08fb30fc9cb6d971999_masked.jpg'

# Load the original image and mask
original_image = cv2.imread(original_image_path)
mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

# Ensure mask is binary (0 or 255)
_, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

# Create a colored mask with the same shape as the original image but filled with the highlighting color, here it's red
colored_mask = np.zeros_like(original_image)
colored_mask[:, :] = [255, 0, 0]  # BGR format, red

# Blend the original image with the colored mask using the binary mask to specify the blending area
alpha = 0.4  # Transparency factor
# Create a masked version of the colored mask, applying it only where the binary mask is white
colored_mask_applied = np.where(binary_mask[:, :, None] == 255, (colored_mask * alpha).astype(np.uint8), 0)
# Add the masked, colored mask to the original image
highlighted_image = cv2.add(original_image, colored_mask_applied)

# Assuming white pixels represent the lesion, count them
white_pixels = np.sum(binary_mask == 255)

# Hypothetical scale factor: Assuming each pixel represents 0.25 square units
scale_factor_per_pixel = 0.25
real_area = white_pixels * scale_factor_per_pixel

# Combine area and units into one string
area_text = f'Area: {real_area} sq units'

# Choose a smaller font size for better fitting
font_scale = 0.7
font_thickness = 2

# Calculate the size of the text box
(text_width, text_height), baseline = cv2.getTextSize(area_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)

# Position the text
text_x, text_y = 10, text_height + 20  # Adjusted to ensure text is visible

# Draw the text on the image
cv2.putText(highlighted_image, area_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

# Convert to RGB to display with PIL (just for visualization here)
highlighted_image_rgb = cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2RGB)

# Display the modified image with PIL
Image.fromarray(highlighted_image_rgb).show()

# Define the filename and extension for saving the image
filename = 'highlighted_image.jpg'  # You can change the filename as needed
save_path = '/Users/shinde/Downloads/segmentation_folder/results/' + filename

# Save the image with the drawn area and contours to the specified path
cv2.imwrite(save_path, highlighted_image)

print(f"Image saved to {save_path}")


Image saved to /Users/shinde/Downloads/segmentation_folder/results/highlighted_image.jpg
